Complete Guide to Effortless ML Monitoring with Evidently.ai

# ML Monitoring Dashboard

In [1]:
!pip install evidently

In [1]:
#Step 1: Import All Necessary Libraries
# Importing necessary modules from Evidently
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric
from evidently.test_suite import TestSuite
from evidently.test_preset import DataQualityTestPreset
from evidently.ui.dashboards import CounterAgg, DashboardPanelCounter, DashboardPanelPlot, PanelValue, PlotType, ReportFilter, DashboardPanelTestSuite, TestFilter, TestSuitePanelType
from evidently.renderers.html_widgets import WidgetSize
from evidently.metric_preset import DataQualityPreset, TargetDriftPreset
from evidently.ui.workspace import Workspace, WorkspaceBase
import pandas as pd

In [3]:
#Step 2: Load the Dataset
# Loading the dataset
df=pd.read_csv("DelayedFlights.csv")

NameError: name 'pd' is not defined

In [ ]:
# Step 3: Define Reference Data and Production Simulation Data
# Defining reference data and production simulation data
reference_data = df[5:7]
prod_simulation_data = df[7:]
batch_size = 2

In [ ]:
# Step 4: Define Workspace and Project Details
# Defining workspace and project details
WORKSPACE = "Guide"
YOUR_PROJECT_NAME = "Evidently Dash"
YOUR_PROJECT_DESCRIPTION = "Learn how to create Evidently Dashboards"

In [ ]:
# Step 5: Create Data Quality Test Suite
# Function to create data quality test suite
def create_data_quality_test_suite(i: int):
    suite = TestSuite(
        tests=[
            DataQualityTestPreset(),
        ],
        timestamp=datetime.datetime.now() + datetime.timedelta(days=i),
        tags = []
    )

    suite.run(reference_data=reference_data, current_data=prod_simulation_data[i * batch_size : (i + 1) * batch_size])
    return suite

In [ ]:
# Step 6: Create a Data Quality Report
# Function to create data quality report
def create_data_quality_report(i: int):
    report = Report(
        metrics=[
            DataQualityPreset(), ColumnDriftMetric(column_name="ArrDelay"),
        ],
        timestamp=datetime.datetime.now() + datetime.timedelta(days=i),
    )

    report.run(reference_data=reference_data, current_data=prod_simulation_data[i * batch_size : (i + 1) * batch_size])
    return report

In [ ]:
# Step 7: Create a Project
# Function to create project
def create_project(workspace: WorkspaceBase):
    project = workspace.create_project(YOUR_PROJECT_NAME)
    project.description = YOUR_PROJECT_DESCRIPTION

    # Adding panels to the dashboard
    project.dashboard.add_panel(
        DashboardPanelCounter(
            filter=ReportFilter(metadata_values={}, tag_values=[]),
            agg=CounterAgg.NONE,
            title="Bank Marketing Dataset",
        )
    )
     project.dashboard.add_panel(
        DashboardPanelPlot(
            title="Target Drift",
            filter=ReportFilter(metadata_values={}, tag_values=[]),
            values=[
                PanelValue(
                    metric_id="ColumnDriftMetric",
                    metric_args={"column_name.name": "ArrDelay"},
                    field_path=ColumnDriftMetric.fields.drift_score,
                    legend="target: ArrDelay",
                ),
            ],
            plot_type=PlotType.LINE,
            size=WidgetSize.HALF
        )
    )
    # Adding test suites to the dashboard
    project.dashboard.add_panel(
        DashboardPanelTestSuite(
            title="All tests: aggregated",
            filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=True),
            size=WidgetSize.HALF,
            time_agg="1M",
        )
    )

    project.dashboard.add_panel(
        DashboardPanelTestSuite(
            title="All tests: detailed",
            filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=True),
            size=WidgetSize.HALF,
            panel_type=TestSuitePanelType.DETAILED,
            time_agg="1D",
        )
    )

    # Saving the project
    project.save()
    return project    
    


In [ ]:
# Step 8: Create a Workspace and Add Reports to the Workspace
# Function to create demo project
def create_demo_project(workspace: str):
    ws = Workspace.create(workspace)
    project = create_project(ws)

    # Adding reports to the workspace
    for i in range(0, 2):
        report = create_data_quality_report(i=i)
        ws.add_report(project.id, report)
        suite = create_data_quality_test_suite(i=i)
        ws.add_report(project.id, suite)

In [ ]:
# Step 9: Call the Main Function
# Main function
if __name__ == "__main__":
    create_demo_project(WORKSPACE)